#### NSMC (NAVER Sentiment Movie Corpus) 감성분류

CNN을 이용한 감성분류

DNN, RNN, LSTM, GRU, CNN 분류 모델 비교

In [1]:
import warnings 
warnings.filterwarnings(action='ignore')

In [2]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
#!pip install konlpy
from konlpy.tag import Okt

### 1) 네이버 영화 리뷰 데이터 로드하기

In [4]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from tqdm import tqdm

In [5]:
df=pd.read_csv("https://raw.githubusercontent.com/hongsukyi/Lectures/main/data/naver_movie_train.txt", sep="\t")

In [6]:
df.head(3)


,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


원본 데이터는 150,000개이나 코랩 교육용은 5,000개로 크기를 줄였다.

In [7]:
print('원본 샘플 개수:',len(df))

원본 샘플 개수: 150000


중복 유무 확인하고 중복 제거

In [8]:
# document 열과 label 열의 중복을 제외한 값의 개수
df['document'].nunique(), df['label'].nunique()

(146182, 2)

In [9]:
df.drop_duplicates(subset=['document'], inplace=True)
print('중복 제거 이후 샘플 개수:',len(df))

중복 제거 이후 샘플 개수: 146183


- df에서 리뷰의 긍, 부정 유무가 기재되어있는 레이블(label) 값의 분포

In [ ]:
df['label'].value_counts().plot(kind = 'bar')

In [ ]:
print(df.groupby('label').size().reset_index(name = 'count'))

- 리뷰 중에 Null 값을 가진 샘플이 있는지 확인

In [ ]:
print(df.isnull().values.any())

- 어떤 열에 Null 값을 가진 샘풀이 존재하는지 확인하자

In [ ]:
print(df.isnull().sum())

- document 열에서 Null 값을 가진 샘플이 총 1개가 존재한다
- Null 값을 가진 샘플이 어느 인덱스의 위치에 존재하는지 확인하자

In [ ]:
df.loc[df.document.isnull()]

In [ ]:
df = df.dropna(how = 'any') # Null 값이 존재하는 행 제거
print(df.isnull().values.any()) # Null 값이 존재하는지 확인

In [ ]:
print(len(df))

### 2)한글 데이터 정제하기 
- 온점(.), ?, 각종 특수문자 제거
- 자음 범위 ㄱ~ㅎ, 모음의 범위 ㅏ ~ ㅣ와 같이 지정.
- 완성형 한글의 범위는 가~힣.
- 한글과 공백을 제외하고 모제 제거하는 정규 표현식을 수행하자.

In [ ]:
# 한글과 공백을 제외하고 모두 제거
df['document'] = df['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","",regex=True)


- 띄어씌기는 유지되면서 온점과 같은 구두점 등은 제거.


In [ ]:
df.head()

 - 리뷰에 공백(whitespace)만 있거나 빈 값을 가진 행이 있다면 Null 값으로 변경

In [ ]:
# white space 데이터를 empty value로 변경
df['document'] = df['document'].str.replace('^ +', "", regex=True) 
df['document'].replace('', np.nan, inplace=True)
print(df.isnull().sum())

- Okt로 Null 값이 789개나 새로 생겼다. 5줄만 출력해보자

In [ ]:
df.loc[df.document.isnull()][:5]

In [ ]:
df = df.dropna(how = 'any')
print(len(df))

### 3) 한글 토근화
불용어 제거, 한국의 조사, 접속사 등 계속해서 추가함

In [ ]:
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

한국어 코큰화를 위한 형태소 분석기는 KoNLPy의 Okt를 사용. 
- stem=True를 사용하면 일정 수준의 정규화를 수행해준다.
- Okt 예제를 보자  

In [ ]:
okt = Okt()
okt.morphs('와 이런 것도 영화라고 차라리 뮤직비디오를 만드는 게 나을 뻔', stem = True)

In [ ]:
df_token = []
for sentence in tqdm(df['document']):
    t_sentence = okt.morphs(sentence, stem=True)   
    removed_sent = [word for word in t_sentence if not word in stopwords] 
    df_token.append(removed_sent)

In [ ]:
df.head()

### 4) 정수 인코딩 
데이터 전처리 과정을 복습해보자.
- 형태소 토큰화 --> 정제 --> 정수 인코딩 --> 패딩 --> 임베딩
- 정수 인코딩을 위해 유효한 BoW 단어가방 크기를 알아보자
- 빈도수가 낮은 단어들을 배제한다. 우선 등장 빈도수 3회 미만을 제거하자.

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df_token)

vocab_size는 20,000이면 충분하다. 

In [ ]:
vocab_size = 20000

In [ ]:
tokenizer = Tokenizer(vocab_size) 
tokenizer.fit_on_texts(df_token)
df_token2 = tokenizer.texts_to_sequences(df_token)

In [ ]:
print(df_token2[:5])

### 5) 빈 샘플(empty samples) 제거 
- 빈(empty) 샘플이 발생. 이유는 빈도수가 낮은 단어만으로 구성되었던 샘플들은 비어 있음.
- 각 샘플들의 길이를 확인해서 길이가 0인 샘플들의 인덱스를 받아오겠습니다.

In [ ]:
drop_train = [index for index, sentence in enumerate(df_token2) if len(sentence) < 1]

In [ ]:
X_train = np.delete(df_text2seq, drop_train, axis=0)
y_train = np.delete(y_train, drop_train, axis=0)
print(len(X_train))

In [ ]:
y_train = np.array(df['label']); y_train[:5]

In [ ]:
# 전체 데이터의 길이는 50으로 맞춘다.
max_len = 50
text2seq_pad = pad_sequences(text2seq, maxlen = max_len)

In [ ]:
text2seq_pad.shape

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(text2seq_pad, y_train, test_size=0.2)

In [ ]:
print(X_train.shape)
print(X_test.shape)

print(y_train.shape)
print(y_test.shape)

#### 2. CNN 등 여러 모델로 NSMC 분류

In [ ]:
from tensorflow.keras.layers import Embedding, Dense, Conv1D, GlobalMaxPooling1D
from tensorflow.keras.models import Sequential

In [ ]:
num_embeddings = 128  
num_hiddens = 64
num_filters = 32 # 커널의 수
kernel_size = 3 # 커널의 크기

In [ ]:
model=Sequential([Embedding(vocab_size, num_embeddings ),
                  Conv1D(num_filters, kernel_size, padding='valid', activation='relu'),
                  GlobalMaxPooling1D(),
                  Dense(num_hiddens, activation='relu'),
                  Dense(1, activation='sigmoid')  ])

In [ ]:
model.compile(optimizer='adam', loss = 'binary_crossentropy', metrics = ['acc'])

In [ ]:
history = model.fit(X_train, y_train, epochs=20, batch_size=100, validation_split=0.23)

In [ ]:
model.summary()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['train', 'val'], loc='upper right')
plt.show()

In [ ]:
# 훈련 정확도 그래프
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['train', 'val'], loc='lower right')
plt.show()

In [ ]:
print("Evaluate on test data")
results = model.evaluate(X_test, y_test)
print("test loss, test acc:", results)

# 3. 리뷰 예측해보기

In [ ]:
import re
def sentiment_predict(new_sentence):
  new_sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','', new_sentence)
  new_sentence = okt.morphs(new_sentence, stem=True) # 토큰화
  new_sentence = [word for word in new_sentence if not word in stopwords] # 불용어 제거
  encoded = tokenizer.texts_to_sequences([new_sentence]) # 정수 인코딩
  pad_new = pad_sequences(encoded, maxlen = max_len) # 패딩
  score = float(model.predict(pad_new)) # 예측
  if(score > 0.5):
    print("{:.2f}% 확률로 긍정 리뷰입니다.\n".format(score * 100))
  else:
    print("{:.2f}% 확률로 부정 리뷰입니다.\n".format((1 - score) * 100))

In [ ]:
sentiment_predict('이 영화 개꿀잼 ㅋㅋㅋ')

In [ ]:
sentiment_predict('이 영화 핵노잼 ㅠㅠ')

In [ ]:
sentiment_predict('이딴게 영화냐 ㅉㅉ')

In [ ]:
sentiment_predict('감독 뭐하는 놈이냐?')